In [10]:
import json
import pymysql
import requests
import sqlalchemy
import sys
from collections import defaultdict
from datetime import datetime, time, timedelta
sys.path.insert(0, '../../../commons/')
sys.path.insert(0, '../../../virasana/')
from ajna_commons.flask.conf import DATABASE, MONGODB_URI
from ajna_commons.flask.conf import VIRASANA_URL, SQL_URI
from bson import ObjectId
from pymongo import MongoClient
from sqlalchemy.orm import sessionmaker


db = MongoClient(host=MONGODB_URI)[DATABASE]
engine = sqlalchemy.create_engine(SQL_URI)

VIRASANA_URL = "https://ajna.labin.rf08.srf/virasana/"
VIRASANA_URL = "https://localhost/virasana/"

In [11]:
diaapesquisar = datetime.today()
datainicial = datetime.strftime(datetime.combine(diaapesquisar, time.min), '%Y-%m-%d  %H:%M:%S')
datafinal = datetime.strftime(datetime.combine(diaapesquisar, time.max), '%Y-%m-%d %H:%M:%S')
print(datainicial, datafinal)

2019-11-22  00:00:00 2019-11-22 23:59:59


In [12]:
def get_conteineres_dia(diaapesquisar):
    datainicial = datetime.strftime(datetime.combine(diaapesquisar, time.min), '%Y-%m-%d  %H:%M:%S')
    datafinal = datetime.strftime(datetime.combine(diaapesquisar, time.max), '%Y-%m-%d %H:%M:%S')
    print(datainicial, datafinal)
    params = {'query': 
          { 'metadata.dataescaneamento': {'$gte': datainicial, '$lte': datafinal},
            'metadata.contentType': 'image/jpeg',
            'metadata.carga': {'$exists': False}
          },
          'projection': 
          {'metadata.numeroinformado': 1,
           'metadata.dataescaneamento': 1}
         }

    r = requests.post(VIRASANA_URL + "/grid_data", json=params, verify=False)
    listacc = list(r.json())
    print(len(listacc))
    listanumerocc = set([item['metadata']['numeroinformado'] for item in listacc])
    print(len(listanumerocc))
    listanumerocc.discard('')
    listanumerocc.discard('ERRO')
    return listanumerocc


In [13]:
listanumerocc = get_conteineres_dia(diaapesquisar)

2019-11-22  00:00:00 2019-11-22 23:59:59
0
0


/home/ivan/pybr/ajna/virasana/virasana-venv/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [20]:
lista =  '("' + '", "'. join(listanumerocc) + '")'

UPDATE_DATAOPERACAO_SQL = \
'UPDATE manifestosresumo'\
' SET dataInicioOperacaoDate = STR_TO_DATE(dataInicioOperacao,"%Y-%m-%d")'\
' WHERE dataInicioOperacaoDate IS NULL AND dataInicioOperacao !="";'

with engine.connect() as cursor:
    sql = sqlalchemy.sql.text(UPDATE_DATAOPERACAO_SQL)
    cursor.execute(sql)


In [269]:
def pesquisa_containers_no_mercante(dia, listanumerocc):
    if len(listanumerocc) == 0:
        return {}, {}
    lista =  '("' + '", "'. join(listanumerocc) + '")'
    SQL_MANIFESTOS = \
    'SELECT numero, idConteinerVazio FROM conteinervazioresumo c '\
    ' inner join manifestosresumo m on c.manifesto = m.numero'\
    ' where tipoTrafego = %s AND '\
    ' dataInicioOperacaoDate >= %s AND dataInicioOperacaoDate <= %s AND '\
    ' c.idConteinerVazio IN ' + lista

    SQL_CONHECIMENTOS = \
    'SELECT c.numeroCEmercante, codigoConteiner FROM itensresumo i'\
    ' inner join conhecimentosresumo c on i.numeroCEmercante = c.numeroCEmercante'\
    ' inner join manifestosresumo m on c.manifestoCE = m.numero'\
    ' WHERE c.tipoBLConhecimento in (\'10\', \'12\') AND'\
    ' m.tipoTrafego = %s AND'\
    ' dataInicioOperacaoDate >= %s AND dataInicioOperacaoDate <= %s AND '\
    ' i.codigoConteiner IN ' + lista
    connection = pymysql.connect(host='mysql.labin.rf08.srf',
                                 user='admmerc',
                                 password='XXXXXX',
                                 db='dbmercante',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    before = diaapesquisar - timedelta(days=5)
    before = datetime.strftime(before, '%Y-%m-%d')
    today = datetime.strftime(diaapesquisar, '%Y-%m-%d')
    after = diaapesquisar + timedelta(days=5)
    after = datetime.strftime(after, '%Y-%m-%d')
    pesquisas_manifesto = [(5, before, today), (7, today, after)]
    manifestos = defaultdict(set)
    conhecimentos = defaultdict(set)
    try:
        with connection.cursor() as cursor:
            #cursor.execute(UPDATE_DATAOPERACAO_SQL)
            for item in pesquisas_manifesto:
                cursor.execute(SQL_MANIFESTOS, item)
                result = cursor.fetchall()
                print('Manifestos %s' % len(result))
                for linha in result:
                    manifestos[linha['idConteinerVazio']].add(linha['numero'])
                cursor.execute(SQL_CONHECIMENTOS, item)
                result = cursor.fetchall()
                print('Conhecimentos %s' % len(result))
                for linha in result:
                    conhecimentos[linha['codigoConteiner']].add(linha['numeroCEmercante'])
    finally:
        connection.close()
    return manifestos, conhecimentos

In [270]:
hoje = datetime.today()
manifestos_acum = {}
conhecimentos_acum = {}
for diasantes in range(11):
    diaapesquisar = hoje - timedelta(days=diasantes)
    listanumerocc = get_conteineres_dia(diaapesquisar)
    manifestos, conhecimentos = pesquisa_containers_no_mercante(diaapesquisar, listanumerocc)
    manifestos_acum.update(manifestos)
    conhecimentos_acum.update(conhecimentos)

2019-11-14  00:00:00 2019-11-14 23:59:59
0
0
2019-11-13  00:00:00 2019-11-13 23:59:59
211
209


c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Manifestos 45
Conhecimentos 8
Manifestos 0
Conhecimentos 20
2019-11-12  00:00:00 2019-11-12 23:59:59


c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


304
303
Manifestos 152
Conhecimentos 11
Manifestos 0
Conhecimentos 83
2019-11-11  00:00:00 2019-11-11 23:59:59


...

928
897
Manifestos 15
Conhecimentos 0
Manifestos 0
Conhecimentos 34
2019-11-05  00:00:00 2019-11-05 23:59:59


c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


742
728
Manifestos 0
Conhecimentos 0
Manifestos 0
Conhecimentos 0
2019-11-04  00:00:00 2019-11-04 23:59:59


c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


468
458
Manifestos 54
Conhecimentos 3
Manifestos 0
Conhecimentos 1


In [271]:
print(manifestos_acum)

{'BMOU9728088': {'1819502228943'}, 'CAIU4309197': {'1819502228943'}, 'CAIU4515578': {'1819502228943'}, 'CAIU5529093': {'1819502228943'}, 'CAIU5532538': {'1819502228951'}, 'CAIU5653519': {'1819502228943'}, 'CAIU7936380': {'1819502228943'}, 'CRXU6918603': {'1819502228943'}, 'CRXU6949923': {'1819502228951'}, 'HJMU6092563': {'1819502228943'}, 'NIDU5128150': {'1819502228943'}, 'NIDU5131391': {'1819502228943'}, 'NIDU5147016': {'1819502228943'}, 'NIDU5151525': {'1819502228943'}, 'NIDU5158073': {'1819502228943'}, 'NIDU5160194': {'1819502228943'}, 'NIDU5164970': {'1819502228943'}, 'NIDU5175830': {'1819502228943'}, 'NIDU5182510': {'1819502228943'}, 'NIDU5197526': {'1819502228943'}, 'NIDU5198862': {'1819502228943'}, 'NIDU5201729': {'1819502228943'}, 'NIDU5210799': {'1819502228943'}, 'NIDU5213463': {'1819502228943'}, 'NIDU5217371': {'1819502228943'}, 'NIDU5235523': {'1819502228943'}, 'NIDU6202801': {'1819502228951'}, 'NIDU6206366': {'1819502228951'}, 'NIDU6206427': {'1819502228943'}, 'NIDU6209920'

In [272]:
print(conhecimentos_acum)


{'MAEU4662316': {'181905232182751'}, 'MORU1326956': {'181905231172310'}, 'SUDU2861521': {'181905232182751'}, 'TLLU1107129': {'181905231171934'}, 'ZCSU2583519': {'181905231440773', '181905241325365'}, 'ZCSU2743324': {'181905231440773', '181905241325365'}, 'APZU4787228': {'181907256492000'}, 'CMAU4195590': {'181907256490563'}, 'CMAU4274787': {'181907256491888'}, 'CMAU5101298': {'181907256492930'}, 'HJMU1910081': {'181907256492930'}, 'MNBU0435671': {'181907254797467'}, 'MNBU0462614': {'181907254797467'}, 'MSKU2551137': {'181907254785450'}, 'TCLU1857697': {'181907256491888'}, 'TCLU6486599': {'181907256492930'}, 'TCNU5810782': {'181907256492930'}, 'TCNU6995850': {'181907256492930'}, 'TEMU1042310': {'181907256492000'}, 'TEMU6481932': {'181907256492930'}, 'TEMU7852745': {'181907256492345'}, 'TGHU6345100': {'181907256492264'}, 'TRLU9452987': {'181907256492000'}, 'TTNU8186517': {'181907256492507'}, 'TTNU8250508': {'181907256489980'}, 'UETU5524337': {'181907256492930'}, 'APHU6240181': {'18190523

In [274]:
for key, value in conhecimentos_acum.items():
    if len(value) > 1:
        print(value)

{'181905231440773', '181905241325365'}
{'181905231440773', '181905241325365'}
{'181905239047647', '181905236775471'}
{'181905243815106', '181905237508728'}
{'181905244865328', '181905237508990'}
{'181905237768983', '181905247446393'}
{'181905248208666', '181905237768207'}
{'151905244918179', '151905245233285'}
{'181905247821070', '181905241554550'}
{'181905243821840', '181905237512083'}
{'181905237509376', '181905244813018'}
{'181905237509376', '181905244813018'}
{'181905243821840', '181905237512083'}
{'181905243774159', '181905237508809'}
{'181905241559358', '181905244899222'}
{'151905245158488', '151905244917520'}
{'151905244231025', '151905239009425'}
{'151907256604702', '151907252227912'}
{'151907256604702', '151907252227912'}
{'151907256604702', '151907252227912'}
{'151905242095345', '151905242576955'}


In [2]:
manifestos_acum = {'BMOU9728088': {'1819502228943'}, 'CAIU4309197': {'1819502228943'}, 'CAIU4515578': {'1819502228943'}, 'CAIU5529093': {'1819502228943'}, 'CAIU5532538': {'1819502228951'}, 'CAIU5653519': {'1819502228943'}, 'CAIU7936380': {'1819502228943'}, 'CRXU6918603': {'1819502228943'}, 'CRXU6949923': {'1819502228951'}, 'HJMU6092563': {'1819502228943'}, 'NIDU5128150': {'1819502228943'}, 'NIDU5131391': {'1819502228943'}, 'NIDU5147016': {'1819502228943'}, 'NIDU5151525': {'1819502228943'}, 'NIDU5158073': {'1819502228943'}, 'NIDU5160194': {'1819502228943'}, 'NIDU5164970': {'1819502228943'}, 'NIDU5175830': {'1819502228943'}, 'NIDU5182510': {'1819502228943'}, 'NIDU5197526': {'1819502228943'}, 'NIDU5198862': {'1819502228943'}, 'NIDU5201729': {'1819502228943'}, 'NIDU5210799': {'1819502228943'}, 'NIDU5213463': {'1819502228943'}, 'NIDU5217371': {'1819502228943'}, 'NIDU5235523': {'1819502228943'}, 'NIDU6202801': {'1819502228951'}, 'NIDU6206366': {'1819502228951'}, 'NIDU6206427': {'1819502228943'}, 'NIDU6209920': {'1819502228951'}, 'NIDU6210052': {'1819502228943'}, 'NIDU6210176': {'1819502228951'}, 'NIDU6500982': {'1819502228951'}, 'NIDU6504268': {'1819502228951'}, 'NIDU6504972': {'1819502228951'}, 'NIDU6504988': {'1819502228943'}, 'NIDU6508155': {'1819502228943'}, 'NIDU6508556': {'1819502228943'}, 'TCLU1288950': {'1819502228951'}, 'TCLU1289581': {'1819502228951'}, 'TCLU1291932': {'1819502228951'}, 'TCLU1294802': {'1819502228943'}, 'TCLU8800517': {'1819502228943'}, 'TRLU1687033': {'1819502228951'}, 'TRLU1698125': {'1819502228943'}, 'AMCU9263844': {'1819502220730'}, 'AMCU9292565': {'1819502220730'}, 'AMCU9296684': {'1819502220730'}, 'APRU5091150': {'1819502220713'}, 'APRU5098323': {'1819502220730'}, 'APRU5105021': {'1819502220721'}, 'APRU5600719': {'1819502220721'}, 'APRU5738668': {'1819502220713'}, 'APRU5749636': {'1819502220713'}, 'APRU5751398': {'1819502220730'}, 'APRU5756281': {'1819502220730'}, 'APRU5766171': {'1819502220713'}, 'APRU5770187': {'1819502220730'}, 'APRU5780842': {'1819502220730'}, 'APRU5787934': {'1819502220730'}, 'APRU5846214': {'1819502220730'}, 'BMOU9704281': {'1819502220713'}, 'BMOU9768265': {'1819502220730'}, 'CAIU5426731': {'1819502220713'}, 'CAIU5556458': {'1819502220713'}, 'CGMU4831303': {'1819502220721'}, 'CGMU4943999': {'1819502220713'}, 'CGMU4963110': {'1819502220730'}, 'CGMU4986331': {'1819502220713'}, 'CGMU5001620': {'1819502220730'}, 'CGMU5023568': {'1819502220730'}, 'CGMU5042001': {'1819502220721'}, 'CGMU5042629': {'1819502220730'}, 'CGMU5064314': {'1819502220721'}, 'CGMU5074544': {'1819502220730'}, 'CGMU5229693': {'1819502220713'}, 'CGMU5243794': {'1819502220713'}, 'CGMU5247151': {'1819502220730'}, 'CGMU5310779': {'1819502220721'}, 'CGMU5335437': {'1819502220721'}, 'CGMU5337342': {'1819502220730'}, 'CGMU6519948': {'1819502220721'}, 'CGMU6529798': {'1819502220730'}, 'CGMU9284981': {'1819502220730'}, 'CGMU9288457': {'1819502220721'}, 'CGMU9294697': {'1819502220730'}, 'CGMU9347890': {'1819502220713'}, 'CGMU9362087': {'1819502220730'}, 'CGMU9370138': {'1819502220730'}, 'CGMU9387568': {'1819502220713'}, 'CRLU1612059': {'1819502220721'}, 'CRLU1612362': {'1819502220713'}, 'CRSU6091724': {'1819502220721'}, 'CRSU6106716': {'1819502220721'}, 'CRSU6114080': {'1819502220713'}, 'CRSU6116680': {'1819502220730'}, 'CRXU6929439': {'1819502220721'}, 'CXRU1362062': {'1819502220730'}, 'CXRU1527028': {'1819502220730'}, 'CXRU1530844': {'1819502220730'}, 'FSCU5652356': {'1819502220713'}, 'GESU9467048': {'1819502220713'}, 'SEGU9157303': {'1819502220730'}, 'SEGU9685321': {'1819502220713'}, 'TCLU1063660': {'1819502220721'}, 'TCLU1139088': {'1819502220730'}, 'TCLU1155622': {'1819502220730'}, 'TCLU1196431': {'1819502220713'}, 'TEMU9225343': {'1819502220730'}, 'TEMU9249160': {'1819502220730'}, 'TGHU9958213': {'1819502220730'}, 'TRIU8055466': {'1819502220730'}, 'TRIU8516146': {'1819502220713'}, 'TRIU8621820': {'1819502220730'}, 'TRIU8659825': {'1819502220730'}, 'TRIU8680057': {'1819502220730'}, 'TRLU1741788': {'1819502220730'}, 'TTNU8236095': {'1819502220730'}, 'TTNU8287873': {'1819502220730'}, 'TTNU8300321': {'1819502220730'}, 'TTNU8651157': {'1819502220713'}, 'APRU5732947': {'1819502220730'}, 'APRU5856064': {'1819502220713'}, 'CGMU5109189': {'1819502220713'}, 'CGMU5162835': {'1819502220721'}, 'CGMU9402683': {'1819502220721'}, 'CRXU6925773': {'1819502220713'}, 'CXRU1361955': {'1819502220721'}, 'CXRU1558564': {'1819502220713'}, 'HLXU3724742': {'1519502125835'}, 'MRKU6207350': {'1519502182812'}, 'MRKU6462735': {'1519502182855'}, 'MRSU4063277': {'1519502182871'}, 'MRSU4077120': {'1519502182812'}, 'PONU7423761': {'1519502182812'}, 'PONU7826242': {'1519502182812'}, 'TTNU8257800': {'1819502220730'}, 'TTNU8308071': {'1819502220713'}, 'APZU4584133': {'1519702276950'}, 'BSIU4002544': {'1519702253594'}, 'BSIU4003540': {'1519702253594'}, 'BSIU4028404': {'1519702253594'}, 'BSIU4054614': {'1519702253594'}, 'BSIU4059730': {'1519702253594'}, 'CLHU4774497': {'1519702253594'}, 'CLHU4782430': {'1519702253594'}, 'CLHU4807646': {'1519702253594'}, 'CMAU8361241': {'1519702276950'}, 'CPSU4034697': {'1519702253594'}, 'DRYU4041567': {'1519702276950'}, 'ECMU4515465': {'1519702276950'}, 'FSCU4608003': {'1519702253594'}, 'FSCU4717508': {'1519702253594'}, 'FSCU4754409': {'1519702253594'}, 'FSCU4801590': {'1519702253594'}, 'FSCU4852637': {'1519702253594'}, 'GATU4346156': {'1519702253594'}, 'GVCU4059494': {'1519702253594'}, 'HLBU1142391': {'1519702253594'}, 'HLBU1196166': {'1519702253594'}, 'HLBU1256028': {'1519702253594'}, 'HLBU1260389': {'1519702253594'}, 'HLBU1265210': {'1519702253594'}, 'HLBU1395569': {'1519702253594'}, 'HLXU5011015': {'1519702253594'}, 'HLXU5054233': {'1519702253594'}, 'HLXU5062558': {'1519702253594'}, 'HLXU5155938': {'1519702253594'}, 'HLXU5156791': {'1519702253594'}, 'HLXU5159912': {'1519702253594'}, 'HLXU5160235': {'1519702253594'}, 'HLXU5171604': {'1519702253594'}, 'HLXU5206383': {'1519702253594'}, 'HLXU5270818': {'1519702253594'}, 'HLXU5276077': {'1519702253594'}, 'HLXU5288993': {'1519702253594'}, 'HLXU5290856': {'1519702253594'}, 'HLXU5295441': {'1519702253594'}, 'HLXU5296176': {'1519702253594'}, 'HLXU5311244': {'1519702253594'}, 'HLXU5328062': {'1519702253594'}, 'HLXU5364460': {'1519702253594'}, 'MAEU6352665': {'1519702259924'}, 'MAEU6362339': {'1519702259924'}, 'MRKU0015637': {'1519702259924'}, 'MRKU0077876': {'1519702259924'}, 'MRKU0087750': {'1519702259924'}, 'MRKU0140388': {'1519702259924'}, 'MRKU0142420': {'1519702259924'}, 'MRKU0149024': {'1519702259924'}, 'MRKU0156635': {'1519702259924'}, 'MRKU0192334': {'1519702259924'}, 'MRKU0205909': {'1519702259924'}, 'MRKU0214304': {'1519702259924'}, 'MRKU0246699': {'1519702259924'}, 'MRKU0270210': {'1519702259924'}, 'MRKU0275017': {'1519702259924'}, 'MRKU0293160': {'1519702259924'}, 'MRKU0305847': {'1519702259924'}, 'MRKU0399506': {'1519702259924'}, 'MRKU0416990': {'1519702259924'}, 'MRKU0458928': {'1519702259924'}, 'MRKU0484953': {'1519702259924'}, 'MRKU0506163': {'1519702259924'}, 'MRKU0615035': {'1519702259924'}, 'MRKU0660986': {'1519702259924'}, 'MRKU0687950': {'1519702259924'}, 'MRKU0690960': {'1519702259924'}, 'MRKU0743247': {'1519702259924'}, 'MRKU0766176': {'1519702259924'}, 'MRKU0777519': {'1519702259924'}, 'MRKU0862966': {'1519702259924'}, 'MRKU0869657': {'1519702259924'}, 'MRKU0935152': {'1519702259924'}, 'MRKU0943060': {'1519702259924'}, 'MRKU0956480': {'1519702259924'}, 'MRKU0991095': {'1519702259924'}, 'MRKU1011214': {'1519702259924'}, 'MRKU1041327': {'1519702259924'}, 'MRKU1072040': {'1519702259924'}, 'MRKU1085686': {'1519702259924'}, 'MRKU1087293': {'1519702259924'}, 'MRKU1090430': {'1519702259924'}, 'MSKU6166761': {'1519702259924'}, 'MSKU6234974': {'1519702259924'}, 'MSKU6279984': {'1519702259924'}, 'MSKU6286915': {'1519702259924'}, 'MSKU6288190': {'1519702259924'}, 'MSKU6315607': {'1519702259924'}, 'MSKU6347559': {'1519702259924'}, 'MSKU6397837': {'1519702259924'}, 'MSKU6399911': {'1519702259924'}, 'MSKU6432624': {'1519702259924'}, 'MSKU6463918': {'1519702259924'}, 'MSKU6473264': {'1519702259924'}, 'MSKU6480160': {'1519702259924'}, 'MSKU6553734': {'1519702259924'}, 'MSKU6600435': {'1519702259924'}, 'MSKU6624587': {'1519702259924'}, 'MSKU6648418': {'1519702259924'}, 'MSKU6653570': {'1519702259924'}, 'MSKU6668245': {'1519702259924'}, 'MSKU6695981': {'1519702259924'}, 'MSKU6720641': {'1519702259924'}, 'MSKU6739631': {'1519702259924'}, 'MSKU6786464': {'1519702259924'}, 'MSKU6824184': {'1519702259924'}, 'MSKU6843271': {'1519702259924'}, 'TCKU4514478': {'1519702253594'}, 'TCLU4083737': {'1519702276950'}, 'TCLU4144920': {'1519702253594'}, 'TCLU4156957': {'1519702253594'}, 'TCLU4158327': {'1519702253594'}, 'TCLU4723346': {'1519702253594'}, 'TCLU5387924': {'1519702253594'}, 'TCLU8987326': {'1519702253594'}, 'TCLU8993653': {'1519702253594'}, 'TGHU4644462': {'1519702253594'}, 'TGHU4892685': {'1519702253594'}, 'TGHU5031300': {'1519702253594'}, 'TGHU5208885': {'1519702253594'}, 'TRLU8644820': {'1519702253594'}, 'TRLU8711933': {'1519702253594'}, 'TRLU9441452': {'1519702253594'}, 'TRLU9451933': {'1519702253594'}, 'TTNU4442755': {'1519702253594'}, 'TTNU4945209': {'1519702253594'}, 'TTNU5963742': {'1519702253594'}, 'UACU8109058': {'1519702253594'}, 'UACU8171521': {'1519702253594'}, 'UACU8184750': {'1519702253594'}, 'UACU8235375': {'1519702253594'}, 'UACU8239180': {'1519702253594'}, 'UACU8254210': {'1519702253594'}, 'UACU8291893': {'1519702253594'}, 'UACU8315552': {'1519702253594'}, 'UACU8334104': {'1519702253594'}, 'UACU8339240': {'1519702253594'}, 'UACU8361970': {'1519702253594'}, 'UACU8368629': {'1519702253594'}, 'UACU8392245': {'1519702253594'}, 'UACU8408402': {'1519702253594'}, 'UACU8552750': {'1519702253594'}, 'UACU8559899': {'1519702253594'}, 'CADU4004899': {'1519702259924'}, 'CLHU4493870': {'1519702259924'}, 'CLHU4507058': {'1519702259924'}, 'CNIU2157364': {'1519702259924'}, 'GRIU4213302': {'1519702259924'}, 'INBU5120353': {'1519702259924'}, 'INBU5209332': {'1519702259924'}, 'MAEU6178804': {'1519702259924'}, 'MAEU6233542': {'1519702259924'}, 'MAEU6294609': {'1519702259924'}, 'MAEU6313273': {'1519702259924'}, 'MAEU6350791': {'1519702259924'}, 'MRKU0008113': {'1519702259924'}, 'MRKU0018298': {'1519702259924'}, 'MRKU0020398': {'1519702259924'}, 'MRKU0026040': {'1519702259924'}, 'MRKU0031200': {'1519702259924'}, 'MRKU0039433': {'1519702259924'}, 'MRKU0044717': {'1519702259924'}, 'MRKU0048137': {'1519702259924'}, 'MRKU0055861': {'1519702259924'}, 'MRKU0056092': {'1519702259924'}, 'MRKU0061397': {'1519702259924'}, 'MRKU0070362': {'1519702259924'}, 'MRKU0078384': {'1519702259924'}, 'MRKU0080083': {'1519702259924'}, 'MRKU0122379': {'1519702259924'}, 'MRKU0127405': {'1519702259924'}, 'MRKU0129774': {'1519702259924'}, 'MRKU0139073': {'1519702259924'}, 'MRKU0141609': {'1519702259924'}, 'MRKU0147166': {'1519702259924'}, 'MRKU0157436': {'1519702259924'}, 'MRKU0161734': {'1519702259924'}, 'MRKU0184107': {'1519702259924'}, 'MRKU0188680': {'1519702259924'}, 'MRKU0189921': {'1519702259924'}, 'MRKU0205344': {'1519702259924'}, 'MRKU0205494': {'1519702259924'}, 'MRKU0206926': {'1519702259924'}, 'MRKU0213247': {'1519702259924'}, 'MRKU0217860': {'1519702259924'}, 'MRKU0218260': {'1519702259924'}, 'MRKU0244280': {'1519702259924'}, 'MRKU0250528': {'1519702259924'}, 'MRKU0250826': {'1519702259924'}, 'MRKU0253949': {'1519702259924'}, 'MRKU0273837': {'1519702259924'}, 'MRKU0274704': {'1519702259924'}, 'MRKU0278340': {'1519702259924'}, 'MRKU0283141': {'1519702259924'}, 'MRKU0295163': {'1519702259924'}, 'MRKU0306720': {'1519702259924'}, 'MRKU0308358': {'1519702259924'}, 'MRKU0310843': {'1519702259924'}, 'MRKU0314962': {'1519702259924'}, 'MRKU0324913': {'1519702259924'}, 'MRKU0332673': {'1519702259924'}, 'MRKU0336956': {'1519702259924'}, 'MRKU0338136': {'1519702259924'}, 'MRKU0343256': {'1519702259924'}, 'MRKU0358298': {'1519702259924'}, 'MRKU0377003': {'1519702259924'}, 'MRKU0384527': {'1519702259924'}, 'MRKU0397885': {'1519702259924'}, 'MRKU0398270': {'1519702259924'}, 'MRKU0406354': {'1519702259924'}, 'MRKU0438520': {'1519702259924'}, 'MRKU0441611': {'1519702259924'}, 'MRKU0444605': {'1519702259924'}, 'MRKU0449510': {'1519702259924'}, 'MRKU0471318': {'1519702259924'}, 'MRKU0472530': {'1519702259924'}, 'MRKU0474180': {'1519702259924'}, 'MRKU0480285': {'1519702259924'}, 'MRKU0485374': {'1519702259924'}, 'MRKU0490468': {'1519702259924'}, 'MRKU0502445': {'1519702259924'}, 'MRKU0504767': {'1519702259924'}, 'MRKU0506369': {'1519702259924'}, 'MRKU0519258': {'1519702259924'}, 'MRKU0536743': {'1519702259924'}, 'MRKU0553123': {'1519702259924'}, 'MRKU0562803': {'1519702259924'}, 'MRKU0594415': {'1519702259924'}, 'MRKU0596296': {'1519702259924'}, 'MRKU0608550': {'1519702259924'}, 'MRKU0610780': {'1519702259924'}, 'MRKU0611533': {'1519702259924'}, 'MRKU0612545': {'1519702259924'}, 'MRKU0627530': {'1519702259924'}, 'MRKU0628053': {'1519702259924'}, 'MRKU0642473': {'1519702259924'}, 'MRKU0645425': {'1519702259924'}, 'MRKU0654617': {'1519702259924'}, 'MRKU0656585': {'1519702259924'}, 'MRKU0665360': {'1519702259924'}, 'MRKU0665756': {'1519702259924'}, 'MRKU0666114': {'1519702259924'}, 'MRKU0691190': {'1519702259924'}, 'MRKU0693830': {'1519702259924'}, 'MRKU0695241': {'1519702259924'}, 'MRKU0696145': {'1519702259924'}, 'MRKU0703630': {'1519702259924'}, 'MRKU0706794': {'1519702259924'}, 'MRKU0711907': {'1519702259924'}, 'MRKU0724843': {'1519702259924'}, 'MRKU0732915': {'1519702259924'}, 'MRKU0737732': {'1519702259924'}, 'MRKU0743822': {'1519702259924'}, 'MRKU0745949': {'1519702259924'}, 'MRKU0747387': {'1519702259924'}, 'MRKU0760291': {'1519702259924'}, 'MRKU0766371': {'1519702259924'}, 'MRKU0768029': {'1519702259924'}, 'MRKU0770860': {'1519702259924'}, 'MRKU0773133': {'1519702259924'}, 'MRKU0774839': {'1519702259924'}, 'MRKU0777458': {'1519702259924'}, 'MRKU0784863': {'1519702259924'}, 'MRKU0784930': {'1519702259924'}, 'MRKU0832529': {'1519702259924'}, 'MRKU0841407': {'1519702259924'}, 'MRKU0876620': {'1519702259924'}, 'MRKU0879167': {'1519702259924'}, 'MRKU0882561': {'1519702259924'}, 'MRKU0886926': {'1519702259924'}, 'MRKU0892554': {'1519702259924'}, 'MRKU0895949': {'1519702259924'}, 'MRKU0898167': {'1519702259924'}, 'MRKU0919249': {'1519702259924'}, 'MRKU0947560': {'1519702259924'}, 'MRKU0961172': {'1519702259924'}, 'MRKU0968958': {'1519702259924'}, 'MRKU0976336': {'1519702259924'}, 'MRKU0980948': {'1519702259924'}, 'MRKU0986078': {'1519702259924'}, 'MRKU1016290': {'1519702259924'}, 'MRKU1017784': {'1519702259924'}, 'MRKU1028198': {'1519702259924'}, 'MRKU1030019': {'1519702259924'}, 'MRKU1031150': {'1519702259924'}, 'MRKU1067609': {'1519702259924'}, 'MRKU1077715': {'1519702259924'}, 'MRKU1103100': {'1519702259924'}, 'MRKU4672310': {'1519702259924'}, 'MRKU4674416': {'1519702259924'}, 'MRKU4682263': {'1519702259924'}, 'MSKU6002449': {'1519702259924'}, 'MSKU6100698': {'1519702259924'}, 'MSKU6112180': {'1519702259924'}, 'MSKU6115928': {'1519702259924'}, 'MSKU6136176': {'1519702259924'}, 'MSKU6175249': {'1519702259924'}, 'MSKU6176899': {'1519702259924'}, 'MSKU6178628': {'1519702259924'}, 'MSKU6192797': {'1519702259924'}, 'MSKU6204244': {'1519702259924'}, 'MSKU6214432': {'1519702259924'}, 'MSKU6228082': {'1519702259924'}, 'MSKU6237778': {'1519702259924'}, 'MSKU6239620': {'1519702259924'}, 'MSKU6253249': {'1519702259924'}, 'MSKU6266544': {'1519702259924'}, 'MSKU6267896': {'1519702259924'}, 'MSKU6300042': {'1519702259924'}, 'MSKU6305846': {'1519702259924'}, 'MSKU6312439': {'1519702259924'}, 'MSKU6324281': {'1519702259924'}, 'MSKU6325879': {'1519702259924'}, 'MSKU6359797': {'1519702259924'}, 'MSKU6363647': {'1519702259924'}, 'MSKU6369856': {'1519702259924'}, 'MSKU6379263': {'1519702259924'}, 'MSKU6391063': {'1519702259924'}, 'MSKU6434606': {'1519702259924'}, 'MSKU6441272': {'1519702259924'}, 'MSKU6446166': {'1519702259924'}, 'MSKU6446377': {'1519702259924'}, 'MSKU6450150': {'1519702259924'}, 'MSKU6461788': {'1519702259924'}, 'MSKU6494225': {'1519702259924'}, 'MSKU6497482': {'1519702259924'}, 'MSKU6506666': {'1519702259924'}, 'MSKU6508925': {'1519702259924'}, 'MSKU6530092': {'1519702259924'}, 'MSKU6541630': {'1519702259924'}, 'MSKU6562926': {'1519702259924'}, 'MSKU6568502': {'1519702259924'}, 'MSKU6571548': {'1519702259924'}, 'MSKU6579786': {'1519702259924'}, 'MSKU6616811': {'1519702259924'}, 'MSKU6622964': {'1519702259924'}, 'MSKU6624458': {'1519702259924'}, 'MSKU6633866': {'1519702259924'}, 'MSKU6695889': {'1519702259924'}, 'MSKU6745465': {'1519702259924'}, 'MSKU6745716': {'1519702259924'}, 'MSKU6747600': {'1519702259924'}, 'MSKU6763411': {'1519702259924'}, 'MSKU6773528': {'1519702259924'}, 'MSKU6780342': {'1519702259924'}, 'MSKU6803999': {'1519702259924'}, 'MSKU6806112': {'1519702259924'}, 'MSKU6807654': {'1519702259924'}, 'MSKU6820300': {'1519702259924'}, 'MSKU6823188': {'1519702259924'}, 'MSKU6824456': {'1519702259924'}, 'MSKU6834829': {'1519702259924'}, 'APHU7275388': {'1519702276950'}, 'BEAU4848300': {'1519702276950'}, 'BMOU2762922': {'1519702244110'}, 'BMOU2883776': {'1519702244110'}, 'BMOU5876410': {'1519702244110'}, 'BSIU2346962': {'1519702244110'}, 'BSIU2542348': {'1519702244110'}, 'BSIU2543030': {'1519702244110'}, 'BSIU2581909': {'1519702244110'}, 'BSIU2741268': {'1519702244110'}, 'BSIU9283594': {'1519702244110'}, 'CAIU4124995': {'1519702244110'}, 'CAIU4143146': {'1519702252687'}, 'CAIU6164240': {'1519702244110'}, 'CAIU7662926': {'1519702244110'}, 'CMAU4531530': {'1519702276950'}, 'CPSU5120850': {'1519702247209'}, 'DFSU1139910': {'1519702244110'}, 'DFSU4229405': {'1519702244110'}, 'ECMU4627655': {'1519702276950'}, 'FCIU5164250': {'1519702244110'}, 'FCIU5560918': {'1519702244110'}, 'FCIU7266878': {'1519702276950'}, 'FCIU9390337': {'1519702244110'}, 'GAOU2137816': {'1519702244110'}, 'GAOU6164044': {'1519702244110'}, 'GCXU2134439': {'1519702244110'}, 'GCXU5017441': {'1519702244110'}, 'GESU7577679': {'1519702252687'}, 'HDMU2629355': {'1519702244110'}, 'HDMU2636035': {'1519702244110'}, 'HDMU2685132': {'1519702244110'}, 'HDMU2750454': {'1519702244110'}, 'HDMU2779329': {'1519702244110'}, 'HDMU6404676': {'1519702244110'}, 'HDMU6671271': {'1519702244110'}, 'HMMU6012061': {'1519702244110'}, 'HMMU6054616': {'1519702244110'}, 'HMMU6154169': {'1519702244110'}, 'INKU6373180': {'1519702244110'}, 'MOFU0635974': {'1519702252687'}, 'PCIU4644275': {'1519702280923'}, 'QIBU1236609': {'1519702254612'}, 'QIBU1241694': {'1519702254612'}, 'SCZU7825052': {'1519702244110'}, 'TCKU4524769': {'1519702252687'}, 'TCLU4685061': {'1519702252687'}, 'TCNU5047990': {'1519702244110'}, 'TCNU5073094': {'1519702244110'}, 'TCNU8268593': {'1519702276950'}, 'TGBU5200244': {'1519702276950'}, 'TGBU5888991': {'1519702244110'}, 'TLLU2362601': {'1519702244110'}, 'TLLU5284689': {'1519702244110'}, 'TRHU2042663': {'1519702244110'}, 'TTNU0602460': {'1519702252687'}, 'UETU4030812': {'1519702252687'}, 'APZU3088139': {'1519502163567'}, 'APZU3803427': {'1519502163567'}, 'BEAU2023574': {'1519502163567'}, 'CAIU2883288': {'1519502132637'}, 'CMAU0260070': {'1519502163567'}, 'CMAU1316873': {'1519502163567'}, 'FBIU0029759': {'1519502163567'}, 'HASU1078545': {'1519502132637'}, 'HASU1169096': {'1519502132637'}, 'HASU1184465': {'1519502132637'}, 'HASU1228413': {'1519502132637'}, 'HASU1479907': {'1519502132637'}, 'MRKU6517458': {'1519502132637'}, 'MRKU6942170': {'1519502132637'}, 'MRKU7269802': {'1519502132637'}, 'MRKU7377638': {'1519502132637'}, 'MRKU7406931': {'1519502132637'}, 'MRKU8322326': {'1519502132637'}, 'MRKU8399966': {'1519502132637'}, 'MRKU9446430': {'1519502132637'}, 'MRKU9565594': {'1519502132637'}, 'MRKU9811698': {'1519502132637'}, 'MRSU0030228': {'1519502132637'}, 'MSKU2827375': {'1519502132637'}, 'MSKU2886254': {'1519502132637'}, 'MSKU2976300': {'1519502132637'}, 'MSKU3104940': {'1519502132637'}, 'MSKU3203199': {'1519502132637'}, 'MSKU3856450': {'1519502132637'}, 'MSKU3959118': {'1519502132637'}, 'MSKU4080714': {'1519502132637'}, 'MSKU4103609': {'1519502132637'}, 'MSKU4228015': {'1519502132637'}, 'MSKU5025970': {'1519502132637'}, 'MSKU5451949': {'1519502132637'}, 'MSKU5579713': {'1519502132637'}, 'MSKU7263719': {'1519502132637'}, 'MSKU7403657': {'1519502132637'}, 'MSKU7486311': {'1519502132637'}, 'MSKU7888917': {'1519502132637'}, 'PONU0471011': {'1519502132637'}, 'PONU0951751': {'1519502132637'}, 'SUDU1831517': {'1519502132637'}, 'SUDU1923742': {'1519502132637'}, 'SUDU7525659': {'1519502132637'}, 'TCKU2250910': {'1519502163567'}, 'TCLU3540510': {'1519502163567'}, 'TCLU6125359': {'1519502132637'}, 'TCLU9919985': {'1519502132637'}, 'TEMU0985330': {'1519502163567'}, 'TEMU2189239': {'1519502132637'}, 'TEMU3552056': {'1519502132637'}, 'TGHU2870408': {'1519502132637'}, 'TGHU2938067': {'1519502132637'}, 'APHU7092216': {'1519702276950'}, 'APZU4523550': {'1519702276950'}, 'BMOU4353461': {'1519702276950'}, 'BMOU6408202': {'1519702276950'}, 'CMAU5177689': {'1519702276950'}, 'CMAU5977850': {'1519702276950'}, 'CMAU7500337': {'1519702276950'}, 'DFSU6255743': {'1519702276950'}, 'ECMU9902195': {'1519702276950'}, 'SEGU5440737': {'1519702276950'}, 'TCLU6536257': {'1519702276950'}, 'TCNU3451810': {'1519702276950'}, 'BMOU9720430': {'1519502134770'}, 'MORU5807219': {'1519502134770'}, 'NYKU7139403': {'1519502134770'}, 'NYKU7941147': {'1519502134770'}, 'SEGU9251480': {'1519502134770'}, 'SZLU9525754': {'1519502134770'}, 'SZLU9705714': {'1519502134770'}, 'TCLU1069838': {'1519502134770'}, 'TCLU1250804': {'1519502134770'}, 'TEMU9020894': {'1519502134770'}, 'TLLU1124980': {'1519502134770'}, 'TRIU8941680': {'1519502134770'}, 'TTNU8106173': {'1519502134770'}, 'TTNU8130396': {'1519502134770'}, 'TTNU8246278': {'1519502134770'}, 'BEAU4189013': {'1519502126971'}, 'BEAU5076161': {'1519502126971'}, 'BHCU4929688': {'1519502126971'}, 'CAIU5455051': {'1519502099567'}, 'CLHU8882416': {'1519502126971'}, 'FCIU8305520': {'1519502126971'}, 'FCIU8775397': {'1519502126971'}, 'FCIU9198366': {'1519502126971'}, 'FSCU8244912': {'1519502126971'}, 'GESU5583902': {'1519502126971'}, 'GESU5696600': {'1519502126971'}, 'GESU5926344': {'1519502126971'}, 'GESU6197479': {'1519502126971'}, 'HAMU1153204': {'1519502126971'}, 'HLBU1081423': {'1519502126971'}, 'HLBU1685995': {'1519502126971'}, 'HLBU2390793': {'1519502126971'}, 'HLBU2511339': {'1519502126971'}, 'HLXU6439559': {'1519502126971'}, 'HLXU6521163': {'1519502126971'}, 'HLXU6591216': {'1519502126971'}, 'HLXU8135730': {'1519502126971'}, 'HLXU8247283': {'1519502126971'}, 'HLXU8290504': {'1519502126971'}, 'HLXU8323439': {'1519502126971'}, 'JXLU5100321': {'1519502099567'}, 'JXLU5816640': {'1519502099567'}, 'TCLU5345277': {'1519502126971'}, 'TCLU5522639': {'1519502126971'}, 'TCNU7469295': {'1519502126971'}, 'TCNU7972479': {'1519502126971'}, 'TCNU8373710': {'1519502126971'}, 'TCNU9960780': {'1519502126971'}, 'TEMU7404041': {'1519502126971'}, 'TGBU6237090': {'1519502126971'}, 'TGHU6494910': {'1519502126971'}, 'TGHU8193095': {'1519502126971'}, 'TGHU8779532': {'1519502126971'}, 'TLLU5402035': {'1519502126971'}, 'TRIU8100312': {'1519502099567'}, 'UACU5472012': {'1519502126971'}, 'UACU5582909': {'1519502126971'}, 'UACU5656956': {'1519502126971'}, 'ZCSU5110309': {'1519502099567'}, 'ZCSU5128041': {'1519502099567'}, 'ZCSU5129428': {'1519502099567'}, 'ZCSU5844771': {'1519502099567'}, 'ZCSU5851240': {'1519502099567'}, 'ZCSU5853217': {'1519502099567'}, 'ZCSU5860284': {'1519502099567'}, 'ZCSU5864886': {'1519502099567'}, 'ZCSU5866786': {'1519502099567'}, 'ZCSU5866997': {'1519502099567'}, 'ZCSU5966614': {'1519502099567'}}

In [1]:
SQL_URI = 'mysql+pymysql://ivan@localhost:3306/mercante'

In [2]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker
from virasana.integracao.mercante.mercantealchemy import Base, Manifesto, ConteinerVazio, \
    Conhecimento, Item, NCMItem, Enumerado

engine = sqlalchemy.create_engine(SQL_URI)
Session = sessionmaker(bind=engine)
session = Session()

session.query(Manifesto).filter(Manifesto.numero == '1319501933641').one()

2019-11-26 10:02:47,007 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  /home/ivan/pybr/ajna/virasana/ajna_commons/flask/error.log
Fazendo log de atividade no arquivo  /home/ivan/pybr/ajna/virasana/ajna_commons/flask/access.log


In [3]:
from sqlalchemy import BigInteger, Column, CHAR, \
    DateTime, func, Integer, Index, MetaData, select, \
    Table, Text, VARCHAR


In [4]:
Enumerado.getTipoManifesto(session, 3)

'lci'

In [5]:
Enumerado.getTipoManifesto(session, 5)

'lce'

In [37]:
from virasana.integracao.mercante.mercante_marshmallow import manifesto_schema, \
    conteinervazio_schema, conhecimento_schema, ncmitem_schema, item_schema

def manifesto_carga(session, manifestos: list, numeroconteiner:str = None):
    dict_carga = {'vazio': True}
    dict_carga['manifesto'] = []
    dict_carga['container'] = []
    for numeromanifesto in manifestos:
        manifesto = session.query(Manifesto).filter(Manifesto.numero == numeromanifesto).one()
        dict_carga['manifesto'].append(manifesto_schema.dump(manifesto, session))
        conteineres = session.query(ConteinerVazio).filter(ConteinerVazio.manifesto == numeromanifesto).all()
        for conteiner in conteineres:
            if numeroconteiner is None:
                dict_carga['container'].append(conteinervazio_schema.dump(conteiner))
            elif conteiner.idConteinerVazio == numeroconteiner:
                dict_carga['container'].append(conteinervazio_schema.dump(conteiner))
    return dict_carga




In [38]:
print(manifesto_carga(session, ['2119501933520'], 'MCAU8570508'))

{'vazio': True, 'manifesto': [{'manifesto': '2119501933520', 'codigoportocarregamento': 'USORF', 'codigoportodescarregamento': 'BRRIG', 'codigoterminalcarregamento': '', 'codigoempresanavegacao': 'DK001046', 'codigoterminaldescarregamento': 'BRRIG005', 'codigoagencianavegacao': '43145945003715', 'tipomanifesto': 'lce'}], 'container': [{'container': 'MCAU8570508', 'tara(kg)': '004680.000'}]}


In [39]:
def conhecimento_carga(session, conhecimentos:list, numeroconteiner:str = None):
    dict_carga = {'vazio': False}
    dict_carga['conhecimento'] = []
    dict_carga['ncm'] = []
    dict_carga['container'] = []
    for numeroconhecimento in conhecimentos:
        conhecimento = session.query(Conhecimento).filter(Conhecimento.numeroCEmercante == numeroconhecimento).one()
        dict_carga['conhecimento'].append(conhecimento_schema.dump(conhecimento))
        itens = session.query(Item).filter(Item.numeroCEmercante == numeroconhecimento).all()
        for item in itens:
            if item.codigoConteiner == numeroconteiner:
                dict_carga['container'].append(item_schema.dump(item))
        ncms = session.query(NCMItem).filter(NCMItem.numeroCEMercante == numeroconhecimento).all()
        for ncmitem in ncms:
            if ncmitem.codigoConteiner == numeroconteiner:
                dict_carga['ncm'].append(ncmitem_schema.dump(ncmitem))
    return dict_carga

In [40]:
conhecimento_carga(session, ['131905215824167'], 'CMAU7454308')

{'vazio': False,
 'conhecimento': [{'conhecimento': '131905215824167',
   'codigoagentenavegacao': '05951386001455',
   'cpfcnpjconsignatario': '67405936000173',
   'cubagem': '0000000076.000',
   'dataemissao': '2019-08-20',
   'descricaomercadoria': 'ALLOY WHEELS',
   'identificacaoembarcador': 'CITIC DICASTAL CO., LTD  NO.185 LONGHAI AVE. ECONOMICAL & TECHNOLOGICAL DEV ZONE,  QINHUANGDAO CITY, HEBEI 066011, CHINA',
   'manifesto': '1319501904706',
   'paisdestino': '',
   'codigoportodestino': 'BRRIO',
   'codigoportoorigem': 'CNXGG',
   'tipo': '12'}],
 'ncm': [{'ncm': '8708',
   'item': '0001',
   'descricao': '',
   'embalagem': '',
   'madeira': '',
   'marca': '',
   'numeroidentificacao': '',
   'qtdevolumes': ''}],
 'container': [{'container': 'CMAU7454308',
   'item': '0001',
   'lacre': 'K1641131',
   'taracontainer': '003700.000',
   'volumeitem': '0000000015.000',
   'pesobrutoitem': '000003395.000',
   'indicadorusoparcial': 'false'}]}